In [5]:
#----------IMPORTS----------
from ultralytics import YOLO
import cv2
import numpy as np

from Libraries.sort import Sort

In [6]:
#----------MODEL AND SORT LOAD----------
Model_Plates = YOLO('YOLOv8/best.pt')
Model_Number = YOLO('Number/best.pt')
tracker = Sort()
id_to_plate = {}
detected_plates = set()

In [7]:
#----------NUMBERS MODEL----------
def get_character_from_class_id(class_id):
    if 0 <= class_id <= 9:
        return chr(ord('0') + class_id)
    elif 10 <= class_id <= 35:
        return chr(ord('A') + (class_id - 10))
    else:
        return ''

In [ ]:
# ----------CAMERA----------
cap = cv2.VideoCapture('IMG_1265.MOV')

# ----------INIT TRACKER----------
tracker = Sort()

# ----------DICTIONARIES----------
id_to_plate_number = {}

# ----------PAUSE STATE----------
is_paused = False

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

bottom_right_area = (frame_width // 2, frame_height // 2, frame_width, frame_height)

current_plate_image = None

# ----------WHILE LOOP----------
while True:
    if not is_paused:
        ret, frame = cap.read()
        if not ret:
            break

        bottom_right_frame = frame[bottom_right_area[1]:bottom_right_area[3], bottom_right_area[0]:bottom_right_area[2]]

        results = Model_Plates(frame)
        detections = []

        #----------CROP PLATES----------
        for result in results[0].boxes:
            class_id = int(result.cls[0])
            if result.xyxy[0].shape[0] >= 4:
                x1, y1, x2, y2 = result.xyxy[0][:4].tolist()
                score = result.conf[0].item()

                if class_id == 1:
                    detections.append([x1, y1, x2, y2, score])

        np_detections = np.array(detections)

        #----------TRACKER----------
        if np_detections.shape[0] > 0:
            tracked_objects = tracker.update(np_detections)

            # ----------READ NUMBERS----------
            tracked_ids = []
            for obj in tracked_objects:
                x1, y1, x2, y2, obj_id = obj
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                tracked_ids.append(obj_id)

                #----------VERIFY BOTTOM RIGHT----------
                if (x1 >= bottom_right_area[0] and y1 >= bottom_right_area[1]):
                    
                    if obj_id not in id_to_plate_number:
                        plate_area = frame[y1:y2, x1:x2]
                        current_plate_image = plate_area
                        
                        #----------ACTIVATE SECOND MODEL
                        number_results = Model_Number(plate_area)
                        plate_text = []

                        char_positions = []

                        for char_result in number_results[0].boxes:
                            char_class_id = int(char_result.cls[0])
                            char = get_character_from_class_id(char_class_id)
                            char_x1, char_y1, char_x2, char_y2 = char_result.xyxy[0][:4].tolist()

                            if (char_x1 >= 0 and char_x2 <= (x2 - x1)) and (char_y1 >= 0 and char_y2 <= (y2 - y1)):
                                char_positions.append((char, char_x1))

                        char_positions.sort(key=lambda x: x[1])
                        full_plate_text = ''.join([c[0] for c in char_positions])
                        id_to_plate_number[obj_id] = full_plate_text

                    #----------SHOW ID----------
                    plate_number = id_to_plate_number[obj_id]
                    text = f"ID: {int(obj_id)} Plate Number: {plate_number}"
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                    cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

            if not any(obj_id in tracked_ids for obj_id in id_to_plate_number.keys()):
                current_plate_image = None

        if current_plate_image is not None:
            plate_resized = cv2.resize(current_plate_image, (200, 100))
            frame[0:100, frame_width - 200:frame_width] = plate_resized

        # ---------- SHOW CAMERA ----------
        frame_resized = cv2.resize(frame, (900, 500))
        cv2.imshow('COIMATRON', frame_resized)

    if cv2.waitKey(1) & 0xFF == ord('z'):
        is_paused = not is_paused

    if is_paused:
        cv2.imshow('COIMATRON', frame_resized)
        while True:
            key = cv2.waitKey(1) & 0xFF
            if key == ord('z'):
                is_paused = False
                break
            elif key == ord('x'):
                cap.release()
                cv2.destroyAllWindows()
                exit()
    if cv2.waitKey(1) & 0xFF == ord('x'):
        break

cap.release()
cv2.destroyAllWindows()


0: 384x640 4 Autos, 2 Placas, 6.5ms
Speed: 3.9ms preprocess, 6.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Autos, 2 Placas, 5.4ms
Speed: 1.9ms preprocess, 5.4ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Autos, 2 Placas, 4.6ms
Speed: 2.0ms preprocess, 4.6ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Autos, 2 Placas, 4.8ms
Speed: 2.7ms preprocess, 4.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Autos, 2 Placas, 7.1ms
Speed: 2.4ms preprocess, 7.1ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Autos, 2 Placas, 4.5ms
Speed: 1.5ms preprocess, 4.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Autos, 2 Placas, 4.9ms
Speed: 2.0ms preprocess, 4.9ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Autos, 2 Placas, 4.5ms
Speed: 1.5ms preprocess, 4.5ms i

KeyboardInterrupt: 

: 